In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from ops import *
import cv2
import os

import numpy as np
np.random.seed(1234)
import matplotlib.pyplot as plt
import tensorflow as tf
import time

In [189]:
global params , criterionGAN

path = r'C:\Users\SAURADIP\Desktop\Ayan Documents\project\code\gans\Shirshendu\Font2Font\New folder\RGB'

params = {'path' : path,
          'batch_size' : 16,
          'output_size': 64,
          'gf_dim': 64,
          'df_dim': 64,
          'L1_lambda': 100,
          'lr': 0.0001,
          'beta1': 0.5,
          'epochs': 200,
          'input_c_dim':1024,
          'output_c_dim':1024,
          'output_height':64,
          'output_width':64,
          'c_dim' :3,
          'z_dim':100
         }

In [190]:
def get_file_paths(path):
    paths = sorted([os.path.join(root, file)  for root, dirs, files in os.walk(path) for file in files])
    return paths[:int(len(paths)*.8)], paths[int(len(paths)*.8):]  #trainpaths, testpaths

def load_data(path):   
    img = cv2.resize(cv2.imread(path),(64,64))
    return img/127.5 - 1.
    

In [191]:
global d_bn1, d_bn2, d_bn3 , g_n1 , g_n2 , g_n3 , g_n4 

global g_bn_e2, g_bn_e3, g_bn_e4, g_bn_e5, g_bn_e6, g_bn_e7, g_bn_e8

global g_bn_d1, g_bn_d2, g_bn_d3, g_bn_d4, g_bn_d5, g_bn_d6 , g_bn_d7

global g_ss_g1 , g_ss_g2 , g_ss_g3 , g_ss_g4 , g_ss_g5 , g_ss_g6 , g_ss_g7 , g_ss_g8 , g_ss_g9 , g_ss_g10

global d_ss_d1 , d_ss_d2 , d_ss_d3 , d_ss_d4 , d_ss_d5 , d_ss_d6 , d_ss_d7 , d_ss_d8 , d_ss_d9 , d_ss_d10

d_bn1 = batch_norm(name='d_bn1')
d_bn2 = batch_norm(name='d_bn2')
d_bn3 = batch_norm(name='d_bn3')

g_bn_e2 = batch_norm(name='g_bn_e2')
g_bn_e3 = batch_norm(name='g_bn_e3')
g_bn_e4 = batch_norm(name='g_bn_e4')
g_bn_e5 = batch_norm(name='g_bn_e5')
g_bn_e6 = batch_norm(name='g_bn_e6')
g_bn_e7 = batch_norm(name='g_bn_e7')
g_bn_e8 = batch_norm(name='g_bn_e8')
g_bn_e9 = batch_norm(name='g_bn_e9')
g_bn_e10 = batch_norm(name='g_bn_e10')
g_bn_e11 = batch_norm(name='g_bn_e11')

g_ss_g1 = batch_norm(name='g_ss_g1')
g_ss_g2 = batch_norm(name='g_ss_g2')
g_ss_g3 = batch_norm(name='g_ss_g3')
g_ss_g4 = batch_norm(name='g_ss_g4')
g_ss_g5 = batch_norm(name='g_ss_g5')
g_ss_g6 = batch_norm(name='g_ss_g6')
g_ss_g7 = batch_norm(name='g_ss_g7')
g_ss_g8 = batch_norm(name='g_ss_g8')
g_ss_g9 = batch_norm(name='g_ss_g9')
g_ss_g10 = batch_norm(name='g_ss_g10')

d_ss_d1 = batch_norm(name='d_ss_d1')
d_ss_d2 = batch_norm(name='d_ss_d2')
d_ss_d3 = batch_norm(name='d_ss_d3')
d_ss_d4 = batch_norm(name='d_ss_d4')
d_ss_d5 = batch_norm(name='d_ss_d5')
d_ss_d6 = batch_norm(name='d_ss_d6')
d_ss_d7 = batch_norm(name='d_ss_d7')
d_ss_d8 = batch_norm(name='d_ss_d8')
d_ss_d9 = batch_norm(name='d_ss_d9')
d_ss_d10 = batch_norm(name='d_ss_d10')



g_n1 = batch_norm(name='g_n1')
g_n2 = batch_norm(name='g_n2')
g_n3 = batch_norm(name='g_n3')
g_n4 = batch_norm(name='g_n4')

In [192]:
def generator(y=None,reuse=False):
    
    s = params['output_size'] # 64
    output_c_dim = 1
    s4, s8, s16, s32 , s64 = int(s/16), int(s/8), int(s/4) , int(s/2) , int(s)
    print(s4)
    gf_dim = params['gf_dim'] # 64
    z_dim= params['z_dim']
    batch_size = params['batch_size']
    # z = random noise
    #z = tf.placeholder(tf.float32, [None, z_dim], name='z')
    z=tf.random_normal([params['batch_size'],z_dim], mean=0.0 , stddev=1.0 )
    print(z.get_shape())

    with tf.variable_scope("generator") as scope:
        if reuse:
            tf.get_variable_scope().reuse_variables()
            
            
        else:
            assert tf.get_variable_scope().reuse == False
        print("-------- Generator ---------")
        # layer 1 size 9 x 9 x 64
        z_, h0_w, h0_b = linear(z, 1024*s4*s4, 'g_h0_lin', with_w=True)
        e1_reshape= tf.reshape(z_,shape=[-1,s4,s4,1024])
        print(e1_reshape.get_shape())
        # layer 1 size 4 x 4 x 1024
        h1= tf.nn.relu(e1_reshape)
        # layer 2 size 8 x 8 x 512
        d1, d1_w, d1_b = deconv2d(h1,[batch_size, s8, s8, 512],k_h=5, k_w=5,d_h=2, d_w=2, name='g_d1', with_w=True)
        e2= g_bn_e3(d1)
        print(e2.get_shape())
        # layer 3 size 16 x 16 x 256
        d1_1 , d1_w_1, d1_b_1 = deconv2d(tf.nn.relu(e2),[batch_size, s16, s16, 256],k_h=5, k_w=5, d_h=2, d_w=2, name='g_d2', with_w=True)
        e3 = g_bn_e4(d1_1)
        print(e3.get_shape())
        # layer 4 size 32 x 32 x 128
        d2 , d2_w, d2_b = deconv2d(tf.nn.relu(e3),[batch_size, s32,s32 ,128],k_h=5, k_w=5, d_h=2, d_w=2, name='g_d3', with_w=True)
        e4 = g_bn_e5(d2)
        print(e4.get_shape())
        # layer 5 size 64 x 64 x 3
        d3 , d3_w, d3_b = deconv2d(tf.nn.relu(e4),[batch_size, s64,s64 ,3],k_h=5, k_w=5, d_h=2, d_w=2, name='g_d4', with_w=True)
        e5 = g_bn_e6(d3)
        print(e5.get_shape())
        
    return tf.nn.tanh(e5)

In [193]:
def discriminator(image , y=None , reuse = False):
    batch_size = params['batch_size']
    df_dim = params['df_dim'] #64
    with tf.variable_scope("discriminator") as scope:
        if reuse:
            tf.get_variable_scope().reuse_variables()
        else:
            assert tf.get_variable_scope().reuse == False
            
        print("-------- Discriminator ---------")
        h0 = lrelu(conv2d(image, df_dim, name='d_h0_conv'))
        print(h0.get_shape())
        h1 = lrelu(d_bn1(conv2d(h0, df_dim*2, name='d_h1_conv')))
        print(h1.get_shape())
        h2 = lrelu(d_bn2(conv2d(h1, df_dim*4, name='d_h2_conv')))
        print(h2.get_shape())
        h3 = lrelu(d_bn3(conv2d(h2, df_dim*8, name='d_h3_conv')))
        print(h3.get_shape())
        h4 = linear(tf.reshape(h3, [batch_size, -1]), 1, 'd_h4_lin')
        print(h4.get_shape())
        return tf.nn.sigmoid(h4) , h4

In [200]:
def DCGAN():
    
    real_img = tf.placeholder(dtype=tf.float32, shape=[params['batch_size'],64,64,3], name = 'real_img') 
    
    fake  = generator()
    D_real , D_real_logits = discriminator(real_img,reuse=False)
    D_fake,D_fake_logits= discriminator(fake,reuse=True) 
    
    
#     d_sum = tf.summary.histogram("d", D_real)
#     d__sum = tf.summary.histogram("d_", D_fake)
    G_sum = tf.summary.image("Generated_Image", fake)
   
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits =D_real_logits, labels= tf.ones_like(D_real)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_fake_logits, labels = tf.zeros_like(D_fake)))
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_fake_logits, labels = tf.ones_like(D_fake)))

    d_loss_real_sum = tf.summary.scalar("d_loss_real", d_loss_real)
    d_loss_fake_sum = tf.summary.scalar("d_loss_fake", d_loss_fake)
                          
    d_loss = d_loss_real + d_loss_fake

    g_loss_sum = tf.summary.scalar("g_loss", g_loss)
    d_loss_sum = tf.summary.scalar("d_loss", d_loss)

    t_vars = tf.trainable_variables()
    d_vars = [var for var in t_vars if 'd_' in var.name]
    g_vars = [var for var in t_vars if 'g_' in var.name]

    #saver = tf.train.Saver()
    
    d_optim = tf.train.AdamOptimizer(0.0005).minimize(d_loss, var_list = d_vars)
    g_optim = tf.train.AdamOptimizer(0.0005).minimize(g_loss, var_list = g_vars)
    
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    summary = tf.summary.merge_all()
    trainwriter = tf.summary.FileWriter("./logs1/train", sess.graph)
    testwriter = tf.summary.FileWriter("./logs1/test", sess.graph)
    
    
    counter = 1
    start_time = time.time()
    
    trainpaths, testpaths = get_file_paths(params['path'])
    
    data_set_size = len(trainpaths)
    
    
    #sess = tf_debug.LocalCLIDebugWrapperSession(sess)
    print (data_set_size//params['batch_size'])
    for epoch in range(params['epochs']):
        
        for idx in range(data_set_size//params['batch_size']):
            
            batch_path = trainpaths[idx * params['batch_size'] : (idx + 1) * params['batch_size']]
            
            batch_data = [load_data(path) for path in batch_path]

            feed_dict = {real_img : batch_data }
            
            _, d_loss_ = sess.run([d_optim, d_loss], feed_dict) # update D network
            _, g_loss_ = sess.run([g_optim, g_loss], feed_dict) # update G network
            _,summary_, g_loss_ = sess.run([g_optim,summary, g_loss], feed_dict) # update G network
            
            trainwriter.add_summary(summary_, counter)
            
            print ('#epoch : ' +str(epoch) + ' idx : ' +str(idx) + ' Dis loss : '+str(d_loss_) + ' Gen loss : '+str(g_loss_))
            
            if idx%10==0:
                
                
                test_summary_ = sess.run(summary, feed_dict)
                testwriter.add_summary(test_summary_,counter)
                _
            counter = counter + 1

In [201]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

global sess

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session()
graph = tf.get_default_graph()



In [ ]:
DCGAN()

4
(16, 100)
-------- Generator ---------
(16, 4, 4, 1024)
(16, 8, 8, 512)
(16, 16, 16, 256)
(16, 32, 32, 128)
(16, 64, 64, 3)
-------- Discriminator ---------
(16, 32, 32, 64)
(16, 16, 16, 128)
(16, 8, 8, 256)
(16, 4, 4, 512)
(16, 1)
-------- Discriminator ---------
(16, 32, 32, 64)
(16, 16, 16, 128)
(16, 8, 8, 256)
(16, 4, 4, 512)
(16, 1)
INFO:tensorflow:Summary name Generated Image is illegal; using Generated_Image instead.
516
#epoch : 0 idx : 0 Dis loss : 1.8633709 Gen loss : 0.00012417139
#epoch : 0 idx : 1 Dis loss : 12.257401 Gen loss : 0.9463855
#epoch : 0 idx : 2 Dis loss : 12.822478 Gen loss : 4.2392206
#epoch : 0 idx : 3 Dis loss : 13.715842 Gen loss : 8.675515
#epoch : 0 idx : 4 Dis loss : 10.648593 Gen loss : 12.572527
#epoch : 0 idx : 5 Dis loss : 5.815179 Gen loss : 13.613386
#epoch : 0 idx : 6 Dis loss : 5.6611648 Gen loss : 5.9638
#epoch : 0 idx : 7 Dis loss : 10.820716 Gen loss : 10.766434
#epoch : 0 idx : 8 Dis loss : 4.999604 Gen loss : 14.941843
#epoch : 0 idx : 9 

#epoch : 0 idx : 126 Dis loss : 0.54858756 Gen loss : 2.9650283
#epoch : 0 idx : 127 Dis loss : 1.1661136 Gen loss : 2.9501824
#epoch : 0 idx : 128 Dis loss : 1.1768069 Gen loss : 3.0583072
#epoch : 0 idx : 129 Dis loss : 0.9599914 Gen loss : 1.8444834
#epoch : 0 idx : 130 Dis loss : 1.7367935 Gen loss : 2.9615831
#epoch : 0 idx : 131 Dis loss : 2.793369 Gen loss : 2.026002
#epoch : 0 idx : 132 Dis loss : 3.5768893 Gen loss : 3.4576037
#epoch : 0 idx : 133 Dis loss : 2.9751909 Gen loss : 4.591098
#epoch : 0 idx : 134 Dis loss : 2.9764013 Gen loss : 4.5808086
#epoch : 0 idx : 135 Dis loss : 4.2097397 Gen loss : 3.094149
#epoch : 0 idx : 136 Dis loss : 2.203956 Gen loss : 1.9637042
#epoch : 0 idx : 137 Dis loss : 4.604866 Gen loss : 2.1866734
#epoch : 0 idx : 138 Dis loss : 2.2885516 Gen loss : 2.8044913
#epoch : 0 idx : 139 Dis loss : 3.1085606 Gen loss : 4.325121
#epoch : 0 idx : 140 Dis loss : 1.747451 Gen loss : 5.3720245
#epoch : 0 idx : 141 Dis loss : 4.15568 Gen loss : 6.8020463
#

#epoch : 0 idx : 258 Dis loss : 1.5616198 Gen loss : 1.3389475
#epoch : 0 idx : 259 Dis loss : 1.0790181 Gen loss : 1.3528324
#epoch : 0 idx : 260 Dis loss : 1.411223 Gen loss : 1.1993471
#epoch : 0 idx : 261 Dis loss : 1.159435 Gen loss : 1.5006998
#epoch : 0 idx : 262 Dis loss : 0.7289605 Gen loss : 2.3708003
#epoch : 0 idx : 263 Dis loss : 1.1460857 Gen loss : 3.3708293
#epoch : 0 idx : 264 Dis loss : 0.6737833 Gen loss : 3.1320257
#epoch : 0 idx : 265 Dis loss : 0.8123439 Gen loss : 4.1771545
#epoch : 0 idx : 266 Dis loss : 0.62630916 Gen loss : 3.991025
#epoch : 0 idx : 267 Dis loss : 0.656735 Gen loss : 3.7297082
#epoch : 0 idx : 268 Dis loss : 0.38558018 Gen loss : 3.9332972
#epoch : 0 idx : 269 Dis loss : 0.8106933 Gen loss : 3.6698995
#epoch : 0 idx : 270 Dis loss : 0.8456427 Gen loss : 2.388855
#epoch : 0 idx : 271 Dis loss : 1.0714333 Gen loss : 1.455245
#epoch : 0 idx : 272 Dis loss : 1.6600926 Gen loss : 0.9998502
#epoch : 0 idx : 273 Dis loss : 2.4340625 Gen loss : 0.7307

#epoch : 0 idx : 389 Dis loss : 1.4762714 Gen loss : 1.5258868
#epoch : 0 idx : 390 Dis loss : 1.2205905 Gen loss : 1.6995888
#epoch : 0 idx : 391 Dis loss : 1.195075 Gen loss : 1.2711015
#epoch : 0 idx : 392 Dis loss : 1.4063151 Gen loss : 1.5243847
#epoch : 0 idx : 393 Dis loss : 1.9693077 Gen loss : 1.1436805
#epoch : 0 idx : 394 Dis loss : 1.3652735 Gen loss : 0.98216534
#epoch : 0 idx : 395 Dis loss : 1.3994497 Gen loss : 0.9317179
#epoch : 0 idx : 396 Dis loss : 1.2549179 Gen loss : 1.175024
#epoch : 0 idx : 397 Dis loss : 1.4571915 Gen loss : 0.9614593
#epoch : 0 idx : 398 Dis loss : 2.6029482 Gen loss : 0.99306417
#epoch : 0 idx : 399 Dis loss : 1.9763056 Gen loss : 1.1452549
#epoch : 0 idx : 400 Dis loss : 1.0652616 Gen loss : 1.0341513
#epoch : 0 idx : 401 Dis loss : 1.1256607 Gen loss : 1.0097985
#epoch : 0 idx : 402 Dis loss : 1.5787826 Gen loss : 1.1021265
#epoch : 0 idx : 403 Dis loss : 1.1871905 Gen loss : 1.2853185
#epoch : 0 idx : 404 Dis loss : 1.1535499 Gen loss : 0.

#epoch : 1 idx : 4 Dis loss : 2.579835 Gen loss : 0.78234106
#epoch : 1 idx : 5 Dis loss : 2.9031844 Gen loss : 0.5566623
#epoch : 1 idx : 6 Dis loss : 2.7006068 Gen loss : 0.35575405
#epoch : 1 idx : 7 Dis loss : 2.5940878 Gen loss : 0.59716165
#epoch : 1 idx : 8 Dis loss : 2.1839983 Gen loss : 0.66123414
#epoch : 1 idx : 9 Dis loss : 1.9430797 Gen loss : 0.87903154
#epoch : 1 idx : 10 Dis loss : 2.7440648 Gen loss : 0.8766761
#epoch : 1 idx : 11 Dis loss : 3.4718175 Gen loss : 0.992617
#epoch : 1 idx : 12 Dis loss : 3.586976 Gen loss : 0.736809
#epoch : 1 idx : 13 Dis loss : 3.989302 Gen loss : 0.91933465
#epoch : 1 idx : 14 Dis loss : 3.0473256 Gen loss : 1.0242127
#epoch : 1 idx : 15 Dis loss : 4.0849667 Gen loss : 0.7528113
#epoch : 1 idx : 16 Dis loss : 6.4757605 Gen loss : 1.1124208
#epoch : 1 idx : 17 Dis loss : 4.759136 Gen loss : 0.70103955
#epoch : 1 idx : 18 Dis loss : 3.6164217 Gen loss : 0.80045784
#epoch : 1 idx : 19 Dis loss : 5.2023993 Gen loss : 1.18356
#epoch : 1 idx

...

#epoch : 6 idx : 437 Dis loss : 2.2770047 Gen loss : 1.1465477
#epoch : 6 idx : 438 Dis loss : 2.1194396 Gen loss : 0.97916996
#epoch : 6 idx : 439 Dis loss : 2.774913 Gen loss : 1.137823
#epoch : 6 idx : 440 Dis loss : 2.5393174 Gen loss : 0.80092806
#epoch : 6 idx : 441 Dis loss : 2.322391 Gen loss : 0.65833
#epoch : 6 idx : 442 Dis loss : 2.031684 Gen loss : 0.5128368
#epoch : 6 idx : 443 Dis loss : 2.3550458 Gen loss : 0.4522369
#epoch : 6 idx : 444 Dis loss : 1.8585179 Gen loss : 0.592597
#epoch : 6 idx : 445 Dis loss : 2.2406862 Gen loss : 0.54179114
#epoch : 6 idx : 446 Dis loss : 2.4878798 Gen loss : 0.5272672
#epoch : 6 idx : 447 Dis loss : 2.0984359 Gen loss : 0.58082587
#epoch : 6 idx : 448 Dis loss : 1.7703097 Gen loss : 0.6466976
#epoch : 6 idx : 449 Dis loss : 2.1634617 Gen loss : 0.65501904
#epoch : 6 idx : 450 Dis loss : 2.150508 Gen loss : 0.5958793
#epoch : 6 idx : 451 Dis loss : 1.7237883 Gen loss : 0.5987141
#epoch : 6 idx : 452 Dis loss : 2.462986 Gen loss : 0.5067

#epoch : 7 idx : 52 Dis loss : 1.2411401 Gen loss : 0.86330855
#epoch : 7 idx : 53 Dis loss : 1.0328801 Gen loss : 0.89128566
#epoch : 7 idx : 54 Dis loss : 0.99785614 Gen loss : 0.7979729
#epoch : 7 idx : 55 Dis loss : 1.0986106 Gen loss : 0.8561878
#epoch : 7 idx : 56 Dis loss : 1.1585083 Gen loss : 0.8709581
#epoch : 7 idx : 57 Dis loss : 0.94013846 Gen loss : 0.91753054
#epoch : 7 idx : 58 Dis loss : 1.2762134 Gen loss : 0.96848494
#epoch : 7 idx : 59 Dis loss : 1.1740594 Gen loss : 0.9034555
#epoch : 7 idx : 60 Dis loss : 1.1341646 Gen loss : 0.99402714
#epoch : 7 idx : 61 Dis loss : 1.368046 Gen loss : 0.95687675
#epoch : 7 idx : 62 Dis loss : 1.2378867 Gen loss : 0.93429047
#epoch : 7 idx : 63 Dis loss : 1.570876 Gen loss : 0.9054507
#epoch : 7 idx : 64 Dis loss : 1.4626143 Gen loss : 0.9626152
#epoch : 7 idx : 65 Dis loss : 2.024512 Gen loss : 0.9057811
#epoch : 7 idx : 66 Dis loss : 2.0105865 Gen loss : 0.8602344
#epoch : 7 idx : 67 Dis loss : 3.0661874 Gen loss : 0.7102319
#e

#epoch : 7 idx : 183 Dis loss : 1.1024027 Gen loss : 0.70386755
#epoch : 7 idx : 184 Dis loss : 1.1627936 Gen loss : 0.75905675
#epoch : 7 idx : 185 Dis loss : 1.2250903 Gen loss : 0.54853207
#epoch : 7 idx : 186 Dis loss : 1.6422002 Gen loss : 0.52590764
#epoch : 7 idx : 187 Dis loss : 1.3989727 Gen loss : 0.84060776
#epoch : 7 idx : 188 Dis loss : 1.4058037 Gen loss : 0.8608434
#epoch : 7 idx : 189 Dis loss : 1.1231294 Gen loss : 1.3365018
#epoch : 7 idx : 190 Dis loss : 1.6283287 Gen loss : 1.1430081
#epoch : 7 idx : 191 Dis loss : 1.941692 Gen loss : 1.2661173
#epoch : 7 idx : 192 Dis loss : 2.1812978 Gen loss : 1.1079564
#epoch : 7 idx : 193 Dis loss : 1.5665244 Gen loss : 1.6107372
#epoch : 7 idx : 194 Dis loss : 1.6612136 Gen loss : 1.2938313
#epoch : 7 idx : 195 Dis loss : 1.4679072 Gen loss : 1.0245508
#epoch : 7 idx : 196 Dis loss : 1.4059277 Gen loss : 1.2832558
#epoch : 7 idx : 197 Dis loss : 2.3844705 Gen loss : 1.1723042
#epoch : 7 idx : 198 Dis loss : 1.5087962 Gen loss 

#epoch : 7 idx : 313 Dis loss : 1.810645 Gen loss : 0.75453544
#epoch : 7 idx : 314 Dis loss : 1.7027504 Gen loss : 0.5316048
#epoch : 7 idx : 315 Dis loss : 1.2450769 Gen loss : 0.96138525
#epoch : 7 idx : 316 Dis loss : 1.5883535 Gen loss : 0.8688111
#epoch : 7 idx : 317 Dis loss : 1.8481493 Gen loss : 0.8460995
#epoch : 7 idx : 318 Dis loss : 1.57636 Gen loss : 0.9768194
#epoch : 7 idx : 319 Dis loss : 1.496108 Gen loss : 0.9591072
#epoch : 7 idx : 320 Dis loss : 1.2644421 Gen loss : 0.968122
#epoch : 7 idx : 321 Dis loss : 1.2577116 Gen loss : 0.82524586
#epoch : 7 idx : 322 Dis loss : 1.4368392 Gen loss : 0.86310697
#epoch : 7 idx : 323 Dis loss : 1.2315488 Gen loss : 0.8779406
#epoch : 7 idx : 324 Dis loss : 1.5946759 Gen loss : 0.68604827
#epoch : 7 idx : 325 Dis loss : 1.5529553 Gen loss : 0.81655574
#epoch : 7 idx : 326 Dis loss : 1.4509888 Gen loss : 0.64452857
#epoch : 7 idx : 327 Dis loss : 1.129786 Gen loss : 0.78808475
#epoch : 7 idx : 328 Dis loss : 1.0225941 Gen loss : 

#epoch : 7 idx : 443 Dis loss : 1.4686116 Gen loss : 0.8846742
#epoch : 7 idx : 444 Dis loss : 1.0465612 Gen loss : 1.3476351
#epoch : 7 idx : 445 Dis loss : 1.370793 Gen loss : 0.8774494
#epoch : 7 idx : 446 Dis loss : 1.8662323 Gen loss : 0.9924085
#epoch : 7 idx : 447 Dis loss : 1.6958966 Gen loss : 0.6991764
#epoch : 7 idx : 448 Dis loss : 1.9951488 Gen loss : 0.5747889
#epoch : 7 idx : 449 Dis loss : 1.9899428 Gen loss : 0.5784928
#epoch : 7 idx : 450 Dis loss : 2.2425478 Gen loss : 0.66061264
#epoch : 7 idx : 451 Dis loss : 2.2567253 Gen loss : 0.44236052
#epoch : 7 idx : 452 Dis loss : 2.2206006 Gen loss : 0.7745961
#epoch : 7 idx : 453 Dis loss : 3.0076323 Gen loss : 0.8131739
#epoch : 7 idx : 454 Dis loss : 2.0320625 Gen loss : 0.83635056
#epoch : 7 idx : 455 Dis loss : 1.889878 Gen loss : 1.0913186
#epoch : 7 idx : 456 Dis loss : 2.0973873 Gen loss : 1.3907276
#epoch : 7 idx : 457 Dis loss : 1.7184765 Gen loss : 1.5417154
#epoch : 7 idx : 458 Dis loss : 2.1961606 Gen loss : 1